 こちらのライブラリのコードを修正してからインストールが必要 (2020.12.15現在)
* https://github.com/okankop/vidaug
<br/>以下手順
* 適当な場所で git clone https://github.com/okankop/vidaug
* こちらのリンクを参考に該当箇所を修正 https://github.com/okankop/vidaug/pull/13/files
* python setup.py sdist && pip install dist/vidaug-0.1.tar.gz

In [63]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from vidaug import augmentors as va
import random
import pandas as pd

In [64]:
folder = "./videos/original"
cropSize = (112, 112)

for curDir, dirs, files in os.walk(folder, topdown=False):
    print(files)
len(files)

['4299707_20180105.avi', '23004146_20181102.avi', '23042542_20181212.avi', '7585185_20160216.avi', '22914550_20180612.avi', '20342556_20160726.avi', '22815211_20180207.avi', '9118019_20180308.avi', '22698708_20190627.avi', '7438575_20150325.avi', '7373004_20160829.avi', '8381170_20150416.avi', '20800579_20160426.avi', '22919120_20180815.avi', '23176514_20191004.avi', '21255948_20170913.avi', '7432941_20161208.avi', '20506101_20170316.avi', '23189459_20190617.avi', '22334155_20160531.avi', '1787134_20170828.avi', '21201397_20180406.avi', '8224701_20190115.avi', '9195421_20151119.avi', '20914719_20180201.avi', '8422149_20161013.avi', '22030993_20150601.avi', '21508544_20170523.avi', '8429292_20190731.avi', '22483705_20170607.avi', '23017866_20181120.avi', '7223050_20161219.avi', '22412373_20160916.avi', '23039449_20181122.avi', '21664644_20160615.avi', '5845805_20150609.avi', '22577993_20170327.avi', '22879241_20180510.avi', '1407089_20150721.avi', '20468781_20160308.avi', '5995030_20150

940

In [65]:
def loadvideo(filename: str) -> np.ndarray:

    if not os.path.exists(filename):
        raise FileNotFoundError(filename)

    capture = cv2.VideoCapture(filename)

    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    v = np.zeros((frame_count, frame_width, frame_height, 3), np.uint8)

    for count in range(frame_count):
        ret, frame = capture.read()
        if not ret:
            raise ValueError("Failed to load frame #{} of {}.".format(count, filename))

        v[count] = frame

#     v = v.transpose((3, 0, 1, 2))

    return v

In [71]:
def transform_video(filename, path, video_aug, df):
    v = loadvideo(path)
    print(path)
    print(video_aug)
    print(v.shape)
    
    sometimes = lambda aug: va.Sometimes(1, aug) # Used to apply augmentor with 100% probability
    if video_aug == "RandomRotate":
        seq = va.Sequential([ # randomly rotates the video with a degree randomly choosen from [-10, 10]  
            sometimes(va.RandomRotate((-15, 15)))# horizontally flip the video with 100% probability
        ])
    elif video_aug == "InvertColor":
        seq = va.Sequential([ 
            sometimes(va.InvertColor())
        ])
    elif video_aug == "GaussianBlur":
        seq = va.Sequential([ 
            sometimes(va.GaussianBlur(3.0))
        ])
    elif video_aug == "HorizontalFlip":
        seq = va.Sequential([ 
            sometimes(va.HorizontalFlip())
        ])
    elif video_aug == "Downsample":
        seq = va.Sequential([ 
            sometimes(va.Downsample(0.7))
        ])
    elif video_aug == "Upsample":
        seq = va.Sequential([ 
            sometimes(va.Upsample(1.3))
        ])
    
    frames  = np.array(seq(v))
    print(frames.shape)
    
    fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    
    output_folder = os.path.join("videos", video_aug)
    if not os.path.exists(output_folder):
            os.mkdir(output_folder)
    
    output_movie = cv2.VideoWriter(os.path.join(output_folder, filename), fourcc, 30, cropSize)
    
    for f in frames:
        output_movie.write(f)
    output_movie.release()
    
    row = df[df['filename'] == filename].copy()

    if row["split"].item() == 'TRAIN':
        row["filename"] = video_aug + "_" + filename
        df = df.append(row)
    
    return df

In [78]:
df = pd.read_csv("FileList.csv", index_col=0)
df.head()

,filename,target,patient_id,counts,rand,split
0,23042542_20181212.avi,0,23042542,1,0.910767,TRAIN
1,20342556_20160726.avi,0,20342556,1,0.677759,TRAIN
2,22815211_20180207.avi,0,22815211,1,0.633463,TRAIN
3,7438575_20150325.avi,0,7438575,1,0.551957,TRAIN
4,7373004_20160829.avi,0,7373004,1,0.887080,TRAIN


In [73]:
for filename in files:
    path = os.path.join(folder, filename)
    
    # 必要でないものはコメントアウト  (21/05/27 : RandomRotateでなぜかエラー..)
    
#     transform_video(path, "RandomRotate")
    df =  transform_video(filename, path, "InvertColor", df)
    df = transform_video(filename, path, "GaussianBlur", df)
    df = transform_video(filename, path, "HorizontalFlip", df)
    df = transform_video(filename, path, "Downsample", df)
    df = transform_video(filename, path, "Upsample", df)

./videos/original/4299707_20180105.avi
InvertColor
(55, 112, 112, 3)
(55, 112, 112, 3)
./videos/original/4299707_20180105.avi
GaussianBlur
(55, 112, 112, 3)
(55, 112, 112, 3)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: FutureWarning: `item` has been deprecated and will be removed in a future version


./videos/original/4299707_20180105.avi
HorizontalFlip
(55, 112, 112, 3)
(55, 112, 112, 3)
./videos/original/4299707_20180105.avi
Downsample
(55, 112, 112, 3)
(38, 112, 112, 3)
./videos/original/4299707_20180105.avi
Upsample
(55, 112, 112, 3)
(71, 112, 112, 3)
./videos/original/23004146_20181102.avi
InvertColor
(39, 112, 112, 3)
(39, 112, 112, 3)
./videos/original/23004146_20181102.avi
GaussianBlur
(39, 112, 112, 3)
(39, 112, 112, 3)
./videos/original/23004146_20181102.avi
HorizontalFlip
(39, 112, 112, 3)
(39, 112, 112, 3)
./videos/original/23004146_20181102.avi
Downsample
(39, 112, 112, 3)
(27, 112, 112, 3)
./videos/original/23004146_20181102.avi
Upsample
(39, 112, 112, 3)
(50, 112, 112, 3)
./videos/original/23042542_20181212.avi
InvertColor
(54, 112, 112, 3)
(54, 112, 112, 3)
./videos/original/23042542_20181212.avi
GaussianBlur
(54, 112, 112, 3)
(54, 112, 112, 3)
./videos/original/23042542_20181212.avi
HorizontalFlip
(54, 112, 112, 3)
(54, 112, 112, 3)
./videos/original/23042542_20181

(74, 112, 112, 3)
./videos/original/22334155_20160531.avi
HorizontalFlip
(74, 112, 112, 3)
(74, 112, 112, 3)
./videos/original/22334155_20160531.avi
Downsample
(74, 112, 112, 3)
(51, 112, 112, 3)
./videos/original/22334155_20160531.avi
Upsample
(74, 112, 112, 3)
(96, 112, 112, 3)
./videos/original/1787134_20170828.avi
InvertColor
(72, 112, 112, 3)
(72, 112, 112, 3)
./videos/original/1787134_20170828.avi
GaussianBlur
(72, 112, 112, 3)
(72, 112, 112, 3)
./videos/original/1787134_20170828.avi
HorizontalFlip
(72, 112, 112, 3)
(72, 112, 112, 3)
./videos/original/1787134_20170828.avi
Downsample
(72, 112, 112, 3)
(50, 112, 112, 3)
./videos/original/1787134_20170828.avi
Upsample
(72, 112, 112, 3)
(93, 112, 112, 3)
./videos/original/21201397_20180406.avi
InvertColor
(95, 112, 112, 3)
(95, 112, 112, 3)
./videos/original/21201397_20180406.avi
GaussianBlur
(95, 112, 112, 3)
(95, 112, 112, 3)
./videos/original/21201397_20180406.avi
HorizontalFlip
(95, 112, 112, 3)
(95, 112, 112, 3)
./videos/origina

(51, 112, 112, 3)
./videos/original/1407089_20150721.avi
HorizontalFlip
(51, 112, 112, 3)
(51, 112, 112, 3)
./videos/original/1407089_20150721.avi
Downsample
(51, 112, 112, 3)
(35, 112, 112, 3)
./videos/original/1407089_20150721.avi
Upsample
(51, 112, 112, 3)
(66, 112, 112, 3)
./videos/original/20468781_20160308.avi
InvertColor
(46, 112, 112, 3)
(46, 112, 112, 3)
./videos/original/20468781_20160308.avi
GaussianBlur
(46, 112, 112, 3)
(46, 112, 112, 3)
./videos/original/20468781_20160308.avi
HorizontalFlip
(46, 112, 112, 3)
(46, 112, 112, 3)
./videos/original/20468781_20160308.avi
Downsample
(46, 112, 112, 3)
(32, 112, 112, 3)
./videos/original/20468781_20160308.avi
Upsample
(46, 112, 112, 3)
(59, 112, 112, 3)
./videos/original/5995030_20150616.avi
InvertColor
(76, 112, 112, 3)
(76, 112, 112, 3)
./videos/original/5995030_20150616.avi
GaussianBlur
(76, 112, 112, 3)
(76, 112, 112, 3)
./videos/original/5995030_20150616.avi
HorizontalFlip
(76, 112, 112, 3)
(76, 112, 112, 3)
./videos/original

...

./videos/original/9197575_20170223.avi
GaussianBlur
(51, 112, 112, 3)
(51, 112, 112, 3)
./videos/original/9197575_20170223.avi
HorizontalFlip
(51, 112, 112, 3)
(51, 112, 112, 3)
./videos/original/9197575_20170223.avi
Downsample
(51, 112, 112, 3)
(35, 112, 112, 3)
./videos/original/9197575_20170223.avi
Upsample
(51, 112, 112, 3)
(66, 112, 112, 3)
./videos/original/5363064_20160226.avi
InvertColor
(98, 112, 112, 3)
(98, 112, 112, 3)
./videos/original/5363064_20160226.avi
GaussianBlur
(98, 112, 112, 3)
(98, 112, 112, 3)
./videos/original/5363064_20160226.avi
HorizontalFlip
(98, 112, 112, 3)
(98, 112, 112, 3)
./videos/original/5363064_20160226.avi
Downsample
(98, 112, 112, 3)
(68, 112, 112, 3)
./videos/original/5363064_20160226.avi
Upsample
(98, 112, 112, 3)
(127, 112, 112, 3)
./videos/original/8993529_20160527.avi
InvertColor
(46, 112, 112, 3)
(46, 112, 112, 3)
./videos/original/8993529_20160527.avi
GaussianBlur
(46, 112, 112, 3)
(46, 112, 112, 3)
./videos/original/8993529_20160527.avi
Ho

./videos/original/22380117_20160715.avi
GaussianBlur
(53, 112, 112, 3)
(53, 112, 112, 3)
./videos/original/22380117_20160715.avi
HorizontalFlip
(53, 112, 112, 3)
(53, 112, 112, 3)
./videos/original/22380117_20160715.avi
Downsample
(53, 112, 112, 3)
(37, 112, 112, 3)
./videos/original/22380117_20160715.avi
Upsample
(53, 112, 112, 3)
(68, 112, 112, 3)
./videos/original/22902415_20180601.avi
InvertColor
(54, 112, 112, 3)
(54, 112, 112, 3)
./videos/original/22902415_20180601.avi
GaussianBlur
(54, 112, 112, 3)
(54, 112, 112, 3)
./videos/original/22902415_20180601.avi
HorizontalFlip
(54, 112, 112, 3)
(54, 112, 112, 3)
./videos/original/22902415_20180601.avi
Downsample
(54, 112, 112, 3)
(37, 112, 112, 3)
./videos/original/22902415_20180601.avi
Upsample
(54, 112, 112, 3)
(70, 112, 112, 3)
./videos/original/4933305_20150430.avi
InvertColor
(57, 112, 112, 3)
(57, 112, 112, 3)
./videos/original/4933305_20150430.avi
GaussianBlur
(57, 112, 112, 3)
(57, 112, 112, 3)
./videos/original/4933305_2015043

./videos/original/8595712_20170322.avi
Downsample
(43, 112, 112, 3)
(30, 112, 112, 3)
./videos/original/8595712_20170322.avi
Upsample
(43, 112, 112, 3)
(55, 112, 112, 3)
./videos/original/20710034_20181219.avi
InvertColor
(69, 112, 112, 3)
(69, 112, 112, 3)
./videos/original/20710034_20181219.avi
GaussianBlur
(69, 112, 112, 3)
(69, 112, 112, 3)
./videos/original/20710034_20181219.avi
HorizontalFlip
(69, 112, 112, 3)
(69, 112, 112, 3)
./videos/original/20710034_20181219.avi
Downsample
(69, 112, 112, 3)
(48, 112, 112, 3)
./videos/original/20710034_20181219.avi
Upsample
(69, 112, 112, 3)
(89, 112, 112, 3)
./videos/original/22213607_20151221.avi
InvertColor
(101, 112, 112, 3)
(101, 112, 112, 3)
./videos/original/22213607_20151221.avi
GaussianBlur
(101, 112, 112, 3)
(101, 112, 112, 3)
./videos/original/22213607_20151221.avi
HorizontalFlip
(101, 112, 112, 3)
(101, 112, 112, 3)
./videos/original/22213607_20151221.avi
Downsample
(101, 112, 112, 3)
(70, 112, 112, 3)
./videos/original/22213607_2

(83, 112, 112, 3)
./videos/original/5051461_20171016.avi
HorizontalFlip
(83, 112, 112, 3)
(83, 112, 112, 3)
./videos/original/5051461_20171016.avi
Downsample
(83, 112, 112, 3)
(58, 112, 112, 3)
./videos/original/5051461_20171016.avi
Upsample
(83, 112, 112, 3)
(107, 112, 112, 3)
./videos/original/22189138_20160203.avi
InvertColor
(57, 112, 112, 3)
(57, 112, 112, 3)
./videos/original/22189138_20160203.avi
GaussianBlur
(57, 112, 112, 3)
(57, 112, 112, 3)
./videos/original/22189138_20160203.avi
HorizontalFlip
(57, 112, 112, 3)
(57, 112, 112, 3)
./videos/original/22189138_20160203.avi
Downsample
(57, 112, 112, 3)
(39, 112, 112, 3)
./videos/original/22189138_20160203.avi
Upsample
(57, 112, 112, 3)
(74, 112, 112, 3)
./videos/original/8527954_20150120.avi
InvertColor
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/8527954_20150120.avi
GaussianBlur
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/8527954_20150120.avi
HorizontalFlip
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/origina

(85, 112, 112, 3)
./videos/original/549493_20190104.avi
HorizontalFlip
(85, 112, 112, 3)
(85, 112, 112, 3)
./videos/original/549493_20190104.avi
Downsample
(85, 112, 112, 3)
(59, 112, 112, 3)
./videos/original/549493_20190104.avi
Upsample
(85, 112, 112, 3)
(110, 112, 112, 3)
./videos/original/581017_20180328.avi
InvertColor
(48, 112, 112, 3)
(48, 112, 112, 3)
./videos/original/581017_20180328.avi
GaussianBlur
(48, 112, 112, 3)
(48, 112, 112, 3)
./videos/original/581017_20180328.avi
HorizontalFlip
(48, 112, 112, 3)
(48, 112, 112, 3)
./videos/original/581017_20180328.avi
Downsample
(48, 112, 112, 3)
(33, 112, 112, 3)
./videos/original/581017_20180328.avi
Upsample
(48, 112, 112, 3)
(62, 112, 112, 3)
./videos/original/22419329_20160831.avi
InvertColor
(53, 112, 112, 3)
(53, 112, 112, 3)
./videos/original/22419329_20160831.avi
GaussianBlur
(53, 112, 112, 3)
(53, 112, 112, 3)
./videos/original/22419329_20160831.avi
HorizontalFlip
(53, 112, 112, 3)
(53, 112, 112, 3)
./videos/original/22419329

In [75]:
# オーグメント用のfilelist
df.to_csv('FileList_Aug.csv')